In [12]:
# coding: utf-8


import pickle
import sys
from datetime import datetime
from multiprocessing import cpu_count

import pandas as pd
import dask.dataframe as dd

import Database_Handler as dh
import Basic_Module as bm

if sys.argv[1] == 'naver':
    # Naver
    data = pickle.load(open('./data/pre_data/stastics/for_statistics_Naver_from_mongodb.pickled', 'rb'))
    data = pd.DataFrame.from_dict(data, orient='index')
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'id'}, inplace=True)
    print('Naver : {}'.format(data.shape))
    extData = data.loc[:, ['id', 'title', 'date', 'press', 'rank', 'category', 'number_of_comment',
                           'number_of_crawled_comment']].copy()
    extData['site'] = pd.Series(['Naver'] * extData.shape[0])

else:
    # Daum
    data = pickle.load(open('./data/pre_data/stastics/for_statistics_daum_from_mongodb.pickled', 'rb'))
    data = pd.DataFrame.from_dict(data, orient='index')
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'id'}, inplace=True)
    print('Daum : {}'.format(data.shape))
    extData = data.loc[:, ['id', 'title', 'date', 'press', 'rank', 'category', 'number_of_comment',
                           'number_of_crawled_comment']].copy()
    extData['site'] = pd.Series(['daum'] * extData.shape[0])


def GetNumberOfCommentInDB(row):
    import Database_Handler as dh
    from bson import ObjectId
    mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
    dbname = 'hy_db'
    useDB = dh.Use_Database(mongodb, dbname)
    commentCollection = dh.Use_Collection(useDB, 'comments')
    info = {'site': row['site'],
            'category': row['category'],
            'date': row['date'],
            'rank': row['rank']}
    commentsForNews = commentCollection.find(info)
    realNumCount = commentsForNews.count()
    site = row['site']
    oid = ObjectId(row['id'])
    if site == 'daum':
        newsCollection = dh.Use_Collection(useDB, 'newsDaum')
    else:
        newsCollection = dh.Use_Collection(useDB, 'newsNaver')
    if realNumCount != row['number_of_crawled_comment']:
        newsCollection.update_one({'_id': oid},
                                  {'$set': {'real_number_of_comment': realNumCount}})
    if row.name % 100 == 0:
        print(row.name)


if __name__ == "__main__":
    start = datetime.now()
    ddf = dd.from_pandas(extData, npartitions=30)
    ddf.apply(GetNumberOfCommentInDB, axis=1, meta = int).compute()
    end = datetime.now()
    print('running time : {}'.format(end - start))

Daum : (9372, 11)


In [13]:
x = extData.loc[0]

In [14]:
GetNumberOfCommentInDB(x)

0


In [15]:
from bson import ObjectId

In [16]:
x = ObjectId(x['id'])

In [18]:
import Database_Handler as dh
from bson import ObjectId
mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDB = dh.Use_Database(mongodb, dbname)
commentCollection = dh.Use_Collection(useDB, 'comments')
newsCollection = dh.Use_Collection(useDB, 'newsDaum')

In [20]:
x2 = newsCollection.find({'_id' : x})

In [23]:
for i in x2:
    print (i)

{'_id': ObjectId('5a2a61bf588c13481c229d1e'), 'category': '뉴스', 'date': '2017.12.07', 'link': 'http://v.media.daum.net/v/20171207100332307', 'press': '세계일보', 'rank': '1', 'title': '"밤이 무섭다"..비아그라 공장 연기에 남성들 부작용 호소', 'mainText': " 주민들은 공장에서 배출된 연기가 '남성이 매우 건강해지는 부작용'을 일으킨다며, 이러한 부작용은 사람에 그치지 않고 개에도 나타난다고 지적했다. 이어 부작용으로 '일상생활에 어려움을 느낄 정도'라며 역학조사가 필요하다고 덧붙였다. 지역에서 간호사로 일하는 한 여성은 “공장 인근을 산책하던 중 강아지가 흥분상태인 것을 확인했다”며 “공장이 환경규제를 준수하는지 확인해야 한다”고 주장했다. 공장은 약 20년 전 마을에 들어선 후 현재까지 치료제를 생산했는데 최근 들어 지역 남성들에게 예상치 못한 영향을 미치고 있다. 시민들 문제 제기에 공장 대변인은 “사랑의 연기는 존재하지 않는다”고 일축하며 “남성들에게서 ‘플라세보효과‘가 나타난 듯하다”고 밝혔다. 한편 공장 성명에도 ‘알 수 없는 큰 힘이 작용한다‘는 생각에 반려견과 이곳을 찾는 남성들이 부쩍 늘었다고 전해진다. 이런 모습에 “건강은 연기가 아닌 운동에서 비롯된 것”이라는 추측이 나오고 있다.  * 플라세보효과는 의사가 효과 없는 가짜 약 또는 꾸며낸 치료법을 환자에게 처방하고, 환자의 긍정적인 믿음으로 인해 병세가 호전되는 현상을 말한다. 이동준 기자 blondie@segye.com\n사진= 선데이타임스 캡처", 'keywords': ['부작용', '비아그라', '아일랜드'], 'number_of_comment': 1093, 'real_number_of_comment': 903, 'site': 'daum'}


In [24]:
x

ObjectId('5a2a61bf588c13481c229d1e')

In [25]:
extData.loc[0]

id                                   5a2a61bf588c13481c229d1e
title                        "밤이 무섭다"..비아그라 공장 연기에 남성들 부작용 호소
date                                               2017.12.07
press                                                    세계일보
rank                                                        1
category                                                   뉴스
number_of_comment                                        1093
number_of_crawled_comment                                 911
site                                                     daum
Name: 0, dtype: object